In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
import re

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [3]:
## Load all data
# frames = []
# for filename in os.listdir("./data/newssources"):
#     frame = pd.read_json("./data/newssources/"+filename, lines=True)
#     frames.append(frame)
# df_raw = pd.concat(frames)

In [4]:
# %store df_raw
%store -r df_raw

In [5]:
len(df_raw["id"].unique())

376201

In [6]:
# data size
print(len(df_raw))
df_raw.head()

376201


platformId  platform                date              updated  type  \
0  1.651751e+30  Facebook 2020-11-22 23:59:49  2020-11-25 18:13:36  link   
1  3.783612e+30  Facebook 2020-11-22 23:59:48  2020-11-25 18:11:01  link   
2  2.759518e+29  Facebook 2020-11-22 23:59:35  2020-11-25 12:17:14  link   
3  4.592339e+29  Facebook 2020-11-22 23:59:35  2020-11-25 18:19:35  link   
4  1.518017e+31  Facebook 2020-11-22 23:59:26  2020-11-25 12:05:12  link   

                                                                                title  \
0            L.A. County suspends outdoor dining at restaurants as coronavirus surges   
1                                                        This is a re-share of a post   
2            L.A. County suspends outdoor dining at restaurants as coronavirus surges   
3  First Floating Ocean Hybrid Platform Can Generate Power From Waves, Wind And Solar   
4    Monkey experiments at Max Planck Institute, Germany | Cruelty Free International   

                        caption  \
0                   latimes.com   
1                           NaN   
2                   latimes.com   
3                    forbes.com   
4  crueltyfreeinternational.org   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [7]:
df_raw.columns

Index(['platformId', 'platform', 'date', 'updated', 'type', 'title', 'caption',
       'description', 'message', 'expandedLinks', 'link', 'postUrl',
       'subscriberCount', 'score', 'media', 'statistics', 'account', 'newId',
       'id', 'videoLengthMS', 'imageText', 'brandedContentSponsor',
       'liveVideoStatus'],
      dtype='object')

### Filtered for specific columns

In [8]:
# Filtered for specific columns
col_names = ['id', 'date', 'updated', 'title', 'caption', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'statistics', 'account']
df_cols = df_raw[col_names]

In [9]:
df_cols[0:1]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,statistics,account
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,"[{'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}, {'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}]",https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.73913,"{'actual': {'likeCount': 9, 'shareCount': 32, 'commentCount': 23, 'loveCount': 0, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 9, 'angryCount': 10, 'thankfulCount': 0, 'careCount': 0}, 'expected': {'likeCount': 6, 'shareCount': 4, 'commentCount': 3, 'loveCount': 3, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 1, 'angryCount': 3, 'thankfulCount': 0, 'careCount': 0}}","{'id': 4619429, 'name': 'Council Member Ed Reece', 'handle': 'edgarreece', 'profileImage': 'https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-1/p200x200/41729566_1855617687861700_8432782541798244352_n.jpg?_nc_cat=106&ccb=2&_nc_sid=dbb9e7&_nc_ohc=BintqqCsj3MAX_7XrAe&_nc_ht=scontent-sea1-1.xx&tp=6&oh=8c2d0a3070a712033f82b1ad071c5ab4&oe=5FE269EF', 'subscriberCount': 2354, 'url': 'https://www.facebook.com/165175133572639', 'platform': 'Facebook', 'platformId': '165175133572639', 'accountType': 'facebook_page', 'pageAdminTopCountry': 'US', 'verified': True}"


### Normalizing Account Column Values as Multiple Columns

In [10]:
## Expanding statistics and account columns
df_acc = df_cols.join(pd.json_normalize(df_cols['account']).add_prefix('account_'))
df_acc[0:1]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,statistics,account,account_id,account_name,account_handle,account_profileImage,account_subscriberCount,account_url,account_platform,account_platformId,account_accountType,account_pageAdminTopCountry,account_verified
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,"[{'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}, {'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}]",https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.73913,"{'actual': {'likeCount': 9, 'shareCount': 32, 'commentCount': 23, 'loveCount': 0, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 9, 'angryCount': 10, 'thankfulCount': 0, 'careCount': 0}, 'expected': {'likeCount': 6, 'shareCount': 4, 'commentCount': 3, 'loveCount': 3, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 1, 'angryCount': 3, 'thankfulCount': 0, 'careCount': 0}}","{'id': 4619429, 'name': 'Council Member Ed Reece', 'handle': 'edgarreece', 'profileImage': 'https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-1/p200x200/41729566_1855617687861700_8432782541798244352_n.jpg?_nc_cat=106&ccb=2&_nc_sid=dbb9e7&_nc_ohc=BintqqCsj3MAX_7XrAe&_nc_ht=scontent-sea1-1.xx&tp=6&oh=8c2d0a3070a712033f82b1ad071c5ab4&oe=5FE269EF', 'subscriberCount': 2354, 'url': 'https://www.facebook.com/165175133572639', 'platform': 'Facebook', 'platformId': '165175133572639', 'accountType': 'facebook_page', 'pageAdminTopCountry': 'US', 'verified': True}",4619429,Council Member Ed Reece,edgarreece,https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-1/p200x200/41729566_1855617687861700_8432782541798244352_n.jpg?_nc_cat=106&ccb=2&_nc_sid=dbb9e7&_nc_ohc=BintqqCsj3MAX_7XrAe&_nc_ht=scontent-sea1-1.xx&tp=6&oh=8c2d0a3070a712033f82b1ad071c5ab4&oe=5FE269EF,2354,https://www.facebook.com/165175133572639,Facebook,165175133572639,facebook_page,US,True


In [11]:
print(len(df_raw["id"].unique()))
print(len(df_acc["id"].unique()))

376201
376201


In [12]:
df_acc.columns

Index(['id', 'date', 'updated', 'title', 'caption', 'expandedLinks', 'link',
       'postUrl', 'subscriberCount', 'score', 'statistics', 'account',
       'account_id', 'account_name', 'account_handle', 'account_profileImage',
       'account_subscriberCount', 'account_url', 'account_platform',
       'account_platformId', 'account_accountType',
       'account_pageAdminTopCountry', 'account_verified'],
      dtype='object')

In [13]:
df_acc = df_acc.drop(columns=['account', 'account_profileImage', 'account_platform', 'account_platformId'])
df_acc[0:1]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,statistics,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,"[{'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}, {'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}]",https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.73913,"{'actual': {'likeCount': 9, 'shareCount': 32, 'commentCount': 23, 'loveCount': 0, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 9, 'angryCount': 10, 'thankfulCount': 0, 'careCount': 0}, 'expected': {'likeCount': 6, 'shareCount': 4, 'commentCount': 3, 'loveCount': 3, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 1, 'angryCount': 3, 'thankfulCount': 0, 'careCount': 0}}",4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True


### Normalizing Statistics Column Values as Multiple Columns

In [14]:
stat_col = df_acc.statistics.apply(lambda x: x['actual'])
df_stat = df_acc.join(pd.json_normalize(stat_col).add_prefix('statistics_'))
df_stat[0:1]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,statistics,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,"[{'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}, {'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}]",https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.73913,"{'actual': {'likeCount': 9, 'shareCount': 32, 'commentCount': 23, 'loveCount': 0, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 9, 'angryCount': 10, 'thankfulCount': 0, 'careCount': 0}, 'expected': {'likeCount': 6, 'shareCount': 4, 'commentCount': 3, 'loveCount': 3, 'wowCount': 2, 'hahaCount': 1, 'sadCount': 1, 'angryCount': 3, 'thankfulCount': 0, 'careCount': 0}}",4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0


In [15]:
df_stat.columns

Index(['id', 'date', 'updated', 'title', 'caption', 'expandedLinks', 'link',
       'postUrl', 'subscriberCount', 'score', 'statistics', 'account_id',
       'account_name', 'account_handle', 'account_subscriberCount',
       'account_url', 'account_accountType', 'account_pageAdminTopCountry',
       'account_verified', 'statistics_likeCount', 'statistics_shareCount',
       'statistics_commentCount', 'statistics_loveCount',
       'statistics_wowCount', 'statistics_hahaCount', 'statistics_sadCount',
       'statistics_angryCount', 'statistics_thankfulCount',
       'statistics_careCount'],
      dtype='object')

In [16]:
df_stat = df_stat.drop(columns=['statistics'])

In [17]:
df_stat[0:1]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,"[{'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}, {'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges', 'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}]",https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.73913,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0


### Adding rows based on 'expandedLinks' column values. One row each for every 'original' (unique) key in the dictionary.

In [18]:
df_stat['expandedLinks'].values[0]

[{'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges',
  'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'},
 {'original': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges',
  'expanded': 'https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges'}]

#### rows with no expandedLinks

In [19]:
len(df_raw[df_raw['expandedLinks'].isnull()])

71

In [20]:
df_raw[df_raw['expandedLinks'].isnull()][['expandedLinks', 'link', 'postUrl']]

,expandedLinks,link,postUrl
2504,NaN,NaN,https://www.facebook.com/groups/1232518956918253/permalink/1652157161621095
4539,NaN,NaN,https://www.facebook.com/groups/1721791731419230/permalink/2748368982094828
6251,NaN,NaN,https://www.facebook.com/groups/218210075390452/permalink/893950491149737
25162,NaN,NaN,https://www.facebook.com/groups/1067900339899078/permalink/3750641441624941
26669,NaN,NaN,https://www.facebook.com/Libertychapelunitedmethodistchurch/posts/3778944255458205
49818,NaN,NaN,https://www.facebook.com/WIFOWUF/posts/3836754129702099
61738,NaN,NaN,https://www.facebook.com/VoA.Burmese.News/posts/10158664583670170
71886,NaN,NaN,https://www.facebook.com/194090487326109/posts/3409522532449539
72056,NaN,NaN,https://www.facebook.com/groups/362351338222543/permalink/396189301505413
83579,NaN,NaN,https://www.facebook.com/groups/216000125600706/permalink/903826130151432


In [21]:
df_exp = df_stat.explode('expandedLinks').reset_index()

In [22]:
print(len(df_exp))
print(len(df_exp.groupby('id').count()))

482616
376201


In [23]:
df_exp_count = df_exp[['expandedLinks', 'id']].groupby('id').count()

In [24]:
df_exp_count.sort_values('expandedLinks', ascending=False)[2600:2605]

,expandedLinks
id,
111599342312,5
111632732378,5
111709269512,5
111678901560,5
111591003300,5


In [25]:
df_raw[df_raw['id'] == 111599342312]['expandedLinks']

41958    [{'original': 'https://rewilding.org/wildlife-governance-reform-where-to-begin/', 'expanded': 'https://rewilding.org/wildlife-governance-reform-where-to-begin/'}, {'original': 'https://www.sltrib.com/news/nation-world/2018/09/11/zinke-seeks-more-state/', 'expanded': 'https://www.sltrib.com/news/nation-world/2018/09/11/zinke-seeks-more-state/'}, {'original': 'https://www.azgfd.com/azgfd-invites-hunters-to-take-arizona-small-game-challenge/', 'expanded': 'https://www.azgfd.com/azgfd-invites-hunters-to-take-arizona-small-game-challenge/'}, {'original': 'https://www.hcn.org/issues/50.15/wildlife-agricultural-interests-steer-colorados-wildlife-management', 'expanded': 'https://www.hcn.org/issues/50.15/wildlife-agricultural-interests-steer-colorados-wildlife-management'}, {'original': 'https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3', 'expanded': 'https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3'}]
Name: 

In [26]:
expanded_link_original = df_exp.expandedLinks.apply(lambda x: x['expanded'] if not pd.isnull(x) else None)

In [27]:
expanded_link_original[0:4]

0                                                      https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges
1                                                      https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges
2    https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social
3                                                                                                                                                          https://reut.rs/39bWCNG
Name: expandedLinks, dtype: object

In [28]:
df_exp = df_exp.rename(columns={'expandedLinks': 'expandedLinksRaw'})

In [29]:
# df_final = df_stat.join(df_exp).drop(columns='expandedLinks')
df_final = df_exp.join(expanded_link_original)

In [30]:
df_final.columns

Index(['index', 'id', 'date', 'updated', 'title', 'caption',
       'expandedLinksRaw', 'link', 'postUrl', 'subscriberCount', 'score',
       'account_id', 'account_name', 'account_handle',
       'account_subscriberCount', 'account_url', 'account_accountType',
       'account_pageAdminTopCountry', 'account_verified',
       'statistics_likeCount', 'statistics_shareCount',
       'statistics_commentCount', 'statistics_loveCount',
       'statistics_wowCount', 'statistics_hahaCount', 'statistics_sadCount',
       'statistics_angryCount', 'statistics_thankfulCount',
       'statistics_careCount', 'expandedLinks'],
      dtype='object')

In [31]:
df_final[df_final['id'] == 111599342312][['id', 'expandedLinksRaw', 'expandedLinks']]

,id,expandedLinksRaw,expandedLinks
165901,111599342312,"{'original': 'https://rewilding.org/wildlife-governance-reform-where-to-begin/', 'expanded': 'https://rewilding.org/wildlife-governance-reform-where-to-begin/'}",https://rewilding.org/wildlife-governance-reform-where-to-begin/
165902,111599342312,"{'original': 'https://www.sltrib.com/news/nation-world/2018/09/11/zinke-seeks-more-state/', 'expanded': 'https://www.sltrib.com/news/nation-world/2018/09/11/zinke-seeks-more-state/'}",https://www.sltrib.com/news/nation-world/2018/09/11/zinke-seeks-more-state/
165903,111599342312,"{'original': 'https://www.azgfd.com/azgfd-invites-hunters-to-take-arizona-small-game-challenge/', 'expanded': 'https://www.azgfd.com/azgfd-invites-hunters-to-take-arizona-small-game-challenge/'}",https://www.azgfd.com/azgfd-invites-hunters-to-take-arizona-small-game-challenge/
165904,111599342312,"{'original': 'https://www.hcn.org/issues/50.15/wildlife-agricultural-interests-steer-colorados-wildlife-management', 'expanded': 'https://www.hcn.org/issues/50.15/wildlife-agricultural-interests-steer-colorados-wildlife-management'}",https://www.hcn.org/issues/50.15/wildlife-agricultural-interests-steer-colorados-wildlife-management
165905,111599342312,"{'original': 'https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3', 'expanded': 'https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3'}",https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3


In [32]:
# verifying that 'original' url has been extracted from allnon-null expandedLinks 

lost = df_final[df_final['expandedLinks'].isnull() & df_final['expandedLinksRaw'].notnull()][['expandedLinks', 'expandedLinksRaw']]
lost

,expandedLinks,expandedLinksRaw


In [33]:
final_cols = ['id', 'date', 'updated', 'title', 'caption', 'expandedLinks', 'link', 'postUrl',
       'subscriberCount', 'score', 'account_id', 'account_name',
       'account_handle', 'account_subscriberCount', 'account_url',
       'account_accountType', 'account_pageAdminTopCountry',
       'account_verified', 'statistics_likeCount', 'statistics_shareCount',
       'statistics_commentCount', 'statistics_loveCount',
       'statistics_wowCount', 'statistics_hahaCount', 'statistics_sadCount',
       'statistics_angryCount', 'statistics_thankfulCount',
       'statistics_careCount']
news_df = df_final[final_cols].drop_duplicates()

In [34]:
news_df.head()

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.739130,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0
2,111678477449,2020-11-23 23:59:58,2020-11-25 18:17:53,Trump Administration To Begin Biden Transition Protocols,npr.org,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,https://www.facebook.com/OccupySpokane/posts/3477341109030911,21775,2.333333,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0
3,111737873253,2020-11-24 23:59:23,2020-11-25 16:15:55,"Russia, eyeing foreign markets, says Sputnik COVID-19 vaccine to cost under $20",reuters.com,https://reut.rs/39bWCNG,https://reut.rs/39bWCNG,https://www.facebook.com/groups/335017933691230/permalink/962329240960093,18498,-21.000000,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0
4,111622415061,2020-11-22 23:59:48,2020-11-25 18:11:01,This is a re-share of a post,NaN,https://m.youtube.com/watch?v=6BsluRkxs78&feature=youtu.be,https://www.facebook.com/100576118560804/posts/100621611889588/,https://www.facebook.com/groups/378361242549568/permalink/1253075498411467,388,-0.750000,12571754,Australia & New Zealand Geoengineering Watch,NaN,388,https://www.facebook.com/groups/378361242549568,facebook_group,NaN,False,103,11,3,15,0,1,0,0,0,0
5,111622415061,2020-11-22 23:59:48,2020-11-25 18:11:01,This is a re-share of a post,NaN,https://www.facebook.com/6442438662/posts/10157599122463663/?d=n,https://www.facebook.com/100576118560804/posts/100621611889588/,https://www.facebook.com/groups/378361242549568/permalink/1253075498411467,388,-0.750000,12571754,Australia & New Zealand Geoengineering Watch,NaN,388,https://www.facebook.com/groups/378361242549568,facebook_group,NaN,False,103,11,3,15,0,1,0,0,0,0


In [35]:
len(news_df)

440477

In [36]:
%store news_df

Stored 'news_df' (DataFrame)


#### No. of Unique Ids/Links

In [37]:
unique_posts_by_id = len(news_df["id"].unique())
unique_posts_by_links = len(news_df["link"].unique())
print("Unique Post Ids: ", unique_posts_by_id)
print("Unique Post Links: ", unique_posts_by_links)

Unique Post Ids:  376201
Unique Post Links:  229526


In [38]:
news_df[news_df['id'] == 111599342312]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount
165901,111599342312,2020-11-22 13:05:34,2020-11-25 00:08:13,NaN,NaN,https://rewilding.org/wildlife-governance-reform-where-to-begin/,https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3,https://www.facebook.com/groups/264941567023535/permalink/1626120937572251,30966,-29.0,4410155,(MHP) ÜLKÜCÜLER ASENALAR ASENA ÜLKÜCÜ BOZKURT BOZKURTLAR ÜLKÜCÜLER (MHP),NaN,30963,https://www.facebook.com/264941567023535,facebook_group,NaN,False,45,19,9,3,0,0,0,0,0,0
165902,111599342312,2020-11-22 13:05:34,2020-11-25 00:08:13,NaN,NaN,https://www.sltrib.com/news/nation-world/2018/09/11/zinke-seeks-more-state/,https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3,https://www.facebook.com/groups/264941567023535/permalink/1626120937572251,30966,-29.0,4410155,(MHP) ÜLKÜCÜLER ASENALAR ASENA ÜLKÜCÜ BOZKURT BOZKURTLAR ÜLKÜCÜLER (MHP),NaN,30963,https://www.facebook.com/264941567023535,facebook_group,NaN,False,45,19,9,3,0,0,0,0,0,0
165903,111599342312,2020-11-22 13:05:34,2020-11-25 00:08:13,NaN,NaN,https://www.azgfd.com/azgfd-invites-hunters-to-take-arizona-small-game-challenge/,https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3,https://www.facebook.com/groups/264941567023535/permalink/1626120937572251,30966,-29.0,4410155,(MHP) ÜLKÜCÜLER ASENALAR ASENA ÜLKÜCÜ BOZKURT BOZKURTLAR ÜLKÜCÜLER (MHP),NaN,30963,https://www.facebook.com/264941567023535,facebook_group,NaN,False,45,19,9,3,0,0,0,0,0,0
165904,111599342312,2020-11-22 13:05:34,2020-11-25 00:08:13,NaN,NaN,https://www.hcn.org/issues/50.15/wildlife-agricultural-interests-steer-colorados-wildlife-management,https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3,https://www.facebook.com/groups/264941567023535/permalink/1626120937572251,30966,-29.0,4410155,(MHP) ÜLKÜCÜLER ASENALAR ASENA ÜLKÜCÜ BOZKURT BOZKURTLAR ÜLKÜCÜLER (MHP),NaN,30963,https://www.facebook.com/264941567023535,facebook_group,NaN,False,45,19,9,3,0,0,0,0,0,0
165905,111599342312,2020-11-22 13:05:34,2020-11-25 00:08:13,NaN,NaN,https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3,https://www.facebook.com/GoodWolfPAC/photos/a.1427974507281374/3510220705723400/?type=3,https://www.facebook.com/groups/264941567023535/permalink/1626120937572251,30966,-29.0,4410155,(MHP) ÜLKÜCÜLER ASENALAR ASENA ÜLKÜCÜ BOZKURT BOZKURTLAR ÜLKÜCÜLER (MHP),NaN,30963,https://www.facebook.com/264941567023535,facebook_group,NaN,False,45,19,9,3,0,0,0,0,0,0


#### Saving news_df to csv

In [39]:
# news_df.to_csv('news_df.csv')

#### Loading NewsGuardTech Ratings

In [40]:
with open("./data/ratings/newsguard-ratings.json") as f:
    ratings = f.read()
ratings_data = json.loads(ratings)

In [41]:
ratings_df = pd.DataFrame(ratings_data.items(), columns=['domain', 'rating'])

In [42]:
ratings_df

,domain,rating
0,cnet.com,100.0
1,digg.com,70.0
2,wsj.com,100.0
3,npr.org,100.0
4,reuters.com,100.0
...,...,...
1068,register-herald.com,92.5
1069,communityimpact.com,100.0
1070,newsleader.com,100.0
1071,scnow.com,100.0


In [43]:
# exp = r'(?i)(.*h?t?tps?://)?(\w*(-->>|:))?(www\.)?([A-Za-z_0-9.-]+).*'
exp = r'(?i)(.*h?t?tps?://)?(www\.)?([A-Za-z_0-9.-]+).*'

In [44]:
news_df['domain'] = news_df.expandedLinks.str.extract(exp, expand=True)[2]

In [45]:
news_df[['expandedLinks', 'domain']]

,expandedLinks,domain
0,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,latimes.com
2,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,npr.org
3,https://reut.rs/39bWCNG,reut.rs
4,https://m.youtube.com/watch?v=6BsluRkxs78&feature=youtu.be,m.youtube.com
5,https://www.facebook.com/6442438662/posts/10157599122463663/?d=n,facebook.com
...,...,...
482611,https://www.newsleader.com/story/magnolia/taste/2020/11/20/deep-frying-turkey-tips-advice-time-temperature-size-safety-thanksgiving/6171599002/,newsleader.com
482612,https://scnow.com/news/national/watch-now-look-inside-massive-christmas-mansion-in-new-jersey-and-more-of-todays-top/collection_ca3251a1-e2ca-5275-acaa-2ad804792e7f.html?utm_campaign=snd-autopilot&utm_medium=social&utm_source=facebook_Morning_News,scnow.com
482613,https://scnow.com/sports/college/fmu-signs-athletes-for-baseball-womens-hoops/article_d37323d4-2de2-11eb-af76-9f1c501d5fb4.html?utm_campaign=snd-autopilot&utm_medium=social&utm_source=facebook_Morning_News,scnow.com
482614,https://scnow.com/sports/high-school/marion-rb-qualiek-crawford-on-quite-a-run-with-swamp-foxes/article_b153ba4e-2de1-11eb-bf7a-7f29b6c61193.html?utm_medium=social&utm_source=facebook&utm_campaign=user-share,scnow.com


In [46]:
# QC
len(news_df[news_df['expandedLinks'].notnull() & news_df['domain'].isnull()])

0

In [47]:
# intersection of newsguardtech api domains and crowdtangle api domains
ratings_domains_set = set(ratings_df['domain'].unique())
ct_domains_set = set(news_df['domain'].unique())

In [48]:
len(ct_domains_set.difference(ratings_domains_set))

5960

todo:
pre-tasks: correct domain column 
1. include ratings column
2. include credible (t/f) column

In [49]:
import pandasql as ps

In [50]:
invalid_expanded_links = ps.sqldf("select id, expandedLinks, domain from news_df where domain not like '%.%'")
# news_df[~news_df["domain"].str.contains('.', na=False)][['id', 'expandedLinks', 'domain']]

In [51]:
invalid_expanded_links.to_csv('invalid_expanded_links.csv')

In [52]:
len(invalid_expanded_links.id.unique())

169

In [53]:
# bad data
df_raw[df_raw['id'] == 111663444165]

,platformId,platform,date,updated,type,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,statistics,account,newId,id,videoLengthMS,imageText,brandedContentSponsor,liveVideoStatus
106214,3.336013e+30,Facebook,2020-11-23 17:48:23,2020-11-25 18:05:59,link,Auto Shop and Yard for Rent,classifieds.ksl.com,"$5,000.00/month Auto Shop and Yard for Rent for rent in North Salt Lake, UT on KSL Classifieds. View a wide selection of Commercial Property for Sale or Rent and other great items on KSL Classifieds.",NaN,"[{'original': '$5,000.00/month', 'expanded': '$5,000.00/month'}, {'original': 'https://classifieds.ksl.com/listing/62058651#.X7vwprearT4.facebook', 'expanded': 'https://classifieds.ksl.com/listing/62058651#.X7vwprearT4.facebook'}]",https://classifieds.ksl.com/listing/62058651#.X7vwprearT4.facebook,https://www.facebook.com/groups/333601286811707/permalink/1616837171821439,1544,-4.0,"[{'type': 'photo', 'url': 'https://external-sea1-1.xx.fbcdn.net/safe_image.php?d=AQAP8zbjPerIBZ10&w=600&h=600&url=https%3A%2F%2Fimg.ksl.com%2Fmx%2Fmplace-classifieds.ksl.com%2F911708-1605894818-148383.jpg&cfs=1&_nc_cb=1&_nc_hash=AQB2WTZibGC45La_', 'height': 600, 'width': 600, 'full': 'https://external-sea1-1.xx.fbcdn.net/safe_image.php?d=AQDXp7rSz9oiKh36&url=https%3A%2F%2Fimg.ksl.com%2Fmx%2Fmplace-classifieds.ksl.com%2F911708-1605894818-148383.jpg&_nc_cb=1&_nc_hash=AQBzp2C7giH_ixAm'}]","{'actual': {'likeCount': 0, 'shareCount': 0, 'commentCount': 0, 'loveCount': 0, 'wowCount': 0, 'hahaCount': 0, 'sadCount': 0, 'angryCount': 0, 'thankfulCount': 0, 'careCount': 0}, 'expected': {'likeCount': 1, 'shareCount': 0, 'commentCount': 0, 'loveCount': 1, 'wowCount': 0, 'hahaCount': 0, 'sadCount': 0, 'angryCount': 0, 'thankfulCount': 0, 'careCount': 0}}","{'id': 10058095, 'name': 'Local Utah: A Place For Small Businesses & Events', 'profileImage': 'https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-0/cp0/c23.0.50.50a/p50x50/69259794_10216688129008028_5201240588418023424_n.jpg?_nc_cat=104&ccb=2&_nc_sid=ca434c&_nc_ohc=44dxajmScoMAX9uaPM9&_nc_ht=scontent-sea1-1.xx&tp=27&oh=e4f6b43563957857c5b9f2eed12c129c&oe=5FE5D5DF', 'subscriberCount': 1542, 'url': 'https://www.facebook.com/groups/333601286811707', 'platform': 'Facebook', 'platformId': '333601286811707', 'accountType': 'facebook_group', 'verified': False}",10058095|1616837171821439,111663444165,NaN,NaN,NaN,NaN


In [54]:
invalid_exp_data = df_raw[df_raw['id'].isin(invalid_expanded_links.id.to_list())][['id', 'expandedLinks']]
invalid_exp_data.to_csv('invalid_exp_data.csv')

In [55]:
news_df[(news_df['id'].isin(invalid_expanded_links.id.to_list())) & (news_df['domain'].str.contains('.'))][['id', 'expandedLinks', 'domain']]

,id,expandedLinks,domain
2201,111677826151,http://www.investopedia.com/,investopedia.com
2202,111677826151,http://lifehacker.com/,lifehacker.com
2203,111677826151,https://gibbon.co/,gibbon.co
2204,111677826151,http://duolingo.com/,duolingo.com
2205,111677826151,http://instructables.com/,instructables.com
...,...,...,...
477698,111725334192,https://www.rap-up.com/2020/11/24/grammys-2021-nominations/,rap-up.com
477780,111717909229,http://#therobertdrakesmedianetwork.com/radio/,http
477781,111717909229,https://www.rap-up.com/2020/11/24/iggy-azalea-responds-playboi-carti-tweet/,rap-up.com
478126,111679108428,http://#therobertdrakesmedianetwork.com/radio/,http


In [56]:
news_df[news_df['id'] == 111677826151]

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount,domain
2201,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,http://www.investopedia.com/,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,investopedia.com
2202,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,http://lifehacker.com/,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,lifehacker.com
2203,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,https://gibbon.co/,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,gibbon.co
2204,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,http://duolingo.com/,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,duolingo.com
2205,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,http://instructables.com/,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,instructables.com
2206,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,http://www.fastcompany.com/section/30-second-mba,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,fastcompany.com
2207,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,https://www.coursera.org/,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,coursera.org
2208,111677826151,2020-11-23 23:27:20,2020-11-25 17:56:25,NaN,NaN,http://www.bbc.co.uk/languages,https://www.facebook.com/alwissal3/photos/a.347933192635883/837584393670758/?type=3,https://www.facebook.com/groups/408923716154003/permalink/1255613648151668,50880,-6.5,1959159,Arnold Schwarzenegger - Legenda,ArnoldSchwarzeneggerLegenda,187580,https://www.facebook.com/1413081518987280,facebook_page,CZ,False,0,0,0,0,0,0,0,0,0,0,bbc.co.uk
2209,111677826151,2020-11-23 23:27:20,2020-11-25 17:

#### Expanding short urls to obtain the correct domain

In [57]:
## wherever rating = NaN, get expandedLink - parse it for domain using requests
to_get_expanded_url = news_df[news_df.domain.isin(['trib.al', 'bit.ly', 'tinyurl.com'])][['id', 'expandedLinks', 'domain']]

In [58]:
len(to_get_expanded_url)

3695

In [59]:
short_url_list = to_get_expanded_url.expandedLinks.unique().tolist()

In [60]:
len(short_url_list)

1720

In [61]:
def extract_domain(url):
    exp = r'(?i)(.*h?t?tps?://)?(www\.)?([A-Za-z_0-9.-]+).*'
    try:
        search = re.search(exp, url, re.IGNORECASE)
        return search.groups()[2]
    except Exception as e:
        print(f'Exception for {url} - {e}')
    

In [62]:
def get_extended_url(url):
    extended_url = ''
    domain = ''
    try:
        extended_url = requests.head(url).headers['location']
    except (KeyError, TypeError) as e:
        print(f'Key/Type Exception for {url} - {e}')
    except requests.exceptions.MissingSchema as e:
        print(f'Missing schema for {url}')
        corrected_url =  'http://'+url
        extended_url = requests.head(corrected_url).headers['location']
    except Exception as e:
        print(f'Exception for {url} - {e}')
    domain = extract_domain(extended_url)    
    if domain and domain in ['trib.al', 'bit.ly', 'tinyurl.com']:
        extended_url = requests.head(extended_url).headers['location']
        domain = extract_domain(extended_url)
    return extended_url, domain

In [63]:
def cleanup_url(url):
    pos = url.find('https')
    if url[-1] == '.':
        url = url[0:-1]
    if pos > 0:
        return url[pos:]
    return url

In [64]:
# short_expanded_url_map = {}

In [65]:
get_extended_url('bit.ly/2UeUZGn')

Missing schema for bit.ly/2UeUZGn


('http://indiewritersupport.com/', 'indiewritersupport.com')

In [66]:
# for url in short_url_list:
#     if url not in short_expanded_url_map.keys():
#         expanded_url, domain = get_extended_url(cleanup_url(url))
#         if expanded_url != '':
#             short_expanded_url_map[url] = [expanded_url, domain]

In [67]:
# %store short_expanded_url_map
%store -r short_expanded_url_map

In [68]:
data = [[short_url, val[1]]
        for short_url, val in short_expanded_url_map.items()]
corrected_domain_df = pd.DataFrame(data, columns=['expandedLinks', 'corrected_domain'])

In [69]:
corrected_domain_df

,expandedLinks,corrected_domain
0,https://trib.al/NDSsyrN,npr.org
1,https://trib.al/GhpCOHR,npr.org
2,https://trib.al/ky3cCIP,npr.org
3,https://trib.al/jGUnDPZ,dailymail.co.uk
4,http://bit.ly/2fKbxWg,youtube.com
...,...,...
1703,http://pandemic.https://trib.al/4tbGAbT,wnct.com
1704,https://trib.al/hAuxvhT,blackamericaweb.com
1705,https://trib.al/9LQJm32,blackamericaweb.com
1706,https://trib.al/OxWd4Pr,blackamericaweb.com


In [70]:
get_extended_url('https://bit.ly/3nPNfYa?cc=539458b52df6684692eb0107760bd294')

('https://www.bbc.co.uk/news/world-us-canada-55056570?at_custom1=%5Bpost+type%5D&at_custom4=806AEB6A-2E49-11EB-9DD2-EE373A982C1E&at_custom2=facebook_page&at_medium=custom7&at_campaign=64&at_custom3=BBC+News',
 'bbc.co.uk')

In [71]:
corrected_domain_df[corrected_domain_df.corrected_domain.isin(['trib.al', 'bit.ly', 'tinyurl.com'])][['expandedLinks', 'corrected_domain']]

,expandedLinks,corrected_domain
1554,https://bit.ly/2sVYhB9,bit.ly
1611,https://bit.ly/3lRb2WY,bit.ly
1612,https://bit.ly/3frOutM,bit.ly
1665,https://bit.ly/3fuqlm1,bit.ly


In [72]:
news_updated_domain_df = pd.merge(news_df,corrected_domain_df,how='left',left_on=['expandedLinks'],right_on=['expandedLinks'])

In [73]:
news_updated_domain_df[news_updated_domain_df.corrected_domain.notnull()][['domain', 'corrected_domain']]

,domain,corrected_domain
44,trib.al,npr.org
67,trib.al,npr.org
72,trib.al,npr.org
84,trib.al,dailymail.co.uk
134,trib.al,npr.org
...,...,...
439810,trib.al,blackamericaweb.com
439811,trib.al,blackamericaweb.com
439812,trib.al,blackamericaweb.com
439853,trib.al,blackamericaweb.com


In [74]:
# temp['domain'] = temp[['domain', 'corrected_domain']].apply(lambda x: x[1] if pd.notnull(x[1]) else x[0])
news_updated_domain_df['domain'] = np.where(news_updated_domain_df['corrected_domain'].notnull(), news_updated_domain_df['corrected_domain'], news_updated_domain_df['domain'])

In [75]:
news_updated_domain_df[news_updated_domain_df.corrected_domain.notnull()][['domain', 'corrected_domain']]

,domain,corrected_domain
44,npr.org,npr.org
67,npr.org,npr.org
72,npr.org,npr.org
84,dailymail.co.uk,dailymail.co.uk
134,npr.org,npr.org
...,...,...
439810,blackamericaweb.com,blackamericaweb.com
439811,blackamericaweb.com,blackamericaweb.com
439812,blackamericaweb.com,blackamericaweb.com
439853,blackamericaweb.com,blackamericaweb.com


In [76]:
news_updated_domain_df = news_updated_domain_df.drop(columns=['corrected_domain'])

In [77]:
links404 = news_updated_domain_df[news_updated_domain_df.domain.isin(['trib.al', 'bit.ly', 'tinyurl.com'])][['id', 'expandedLinks', 'domain']]

In [78]:
len(links404)

19

### Joining news_updated_domain_df with ratings_df 

In [79]:
news_ratings_df = pd.merge(news_updated_domain_df,ratings_df,how='left',left_on=['domain'],right_on=['domain'])

In [80]:
# news_ratings_df['rating'] = news_ratings_df['rating'].fillna(-1)
news_ratings_df_filtered = news_ratings_df[news_ratings_df['rating'].notna()]

In [81]:
len(news_ratings_df_filtered)

340931

In [82]:
nonna_ratings = news_ratings_df_filtered.domain.unique()

In [83]:
len(nonna_ratings)

1033

In [84]:
news_ratings_df_filtered.head()

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount,domain,rating
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.739130,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0,latimes.com,100.0
1,111678477449,2020-11-23 23:59:58,2020-11-25 18:17:53,Trump Administration To Begin Biden Transition Protocols,npr.org,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,https://www.facebook.com/OccupySpokane/posts/3477341109030911,21775,2.333333,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0,npr.org,100.0
2,111737873253,2020-11-24 23:59:23,2020-11-25 16:15:55,"Russia, eyeing foreign markets, says Sputnik COVID-19 vaccine to cost under $20",reuters.com,https://reut.rs/39bWCNG,https://reut.rs/39bWCNG,https://www.facebook.com/groups/335017933691230/permalink/962329240960093,18498,-21.000000,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0,reut.rs,100.0
5,111622415061,2020-11-22 23:59:48,2020-11-25 18:11:01,This is a re-share of a post,NaN,https://www.businessinsider.com/china-emotional-surveillance-technology-2018-4,https://www.facebook.com/100576118560804/posts/100621611889588/,https://www.facebook.com/groups/378361242549568/permalink/1253075498411467,388,-0.750000,12571754,Australia & New Zealand Geoengineering Watch,NaN,388,https://www.facebook.com/groups/378361242549568,facebook_group,NaN,False,103,11,3,15,0,1,0,0,0,0,businessinsider.com,100.0
8,111622415061,2020-11-22 23:59:48,2020-11-25 18:11:01,This is a re-share of a post,NaN,https://www.wsj.com/articles/chinas-efforts-to-lead-the-way-in-ai-start-in-its-classrooms-11571958181,https://www.facebook.com/100576118560804/posts/100621611889588/,https://www.facebook.com/groups/378361242549568/permalink/1253075498411467,388,-0.750000,12571754,Australia & New Zealand Geoengineering Watch,NaN,388,https://www.facebook.com/groups/378361242549568,facebook_group,NaN,False,103,11,3,15,0,1,0,0,0,0,wsj.com,100.0


In [85]:
news_ratings_df_filtered['credible'] = news_ratings_df_filtered.rating.apply(lambda x: True if x >= 60 else False)

<ipython-input-85-801278aae509>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_ratings_df_filtered['credible'] = news_ratings_df_filtered.rating.apply(lambda x: True if x >= 60 else False)


In [86]:
news_ratings_df_filtered.head()

,id,date,updated,title,caption,expandedLinks,link,postUrl,subscriberCount,score,account_id,account_name,account_handle,account_subscriberCount,account_url,account_accountType,account_pageAdminTopCountry,account_verified,statistics_likeCount,statistics_shareCount,statistics_commentCount,statistics_loveCount,statistics_wowCount,statistics_hahaCount,statistics_sadCount,statistics_angryCount,statistics_thankfulCount,statistics_careCount,domain,rating,credible
0,111622404459,2020-11-22 23:59:49,2020-11-25 18:13:36,L.A. County suspends outdoor dining at restaurants as coronavirus surges,latimes.com,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.latimes.com/california/story/2020-11-22/l-a-county-suspends-outdoor-dining-at-restaurants-as-coronavirus-surges,https://www.facebook.com/edgarreece/posts/3504433216313464,2351,3.739130,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0,latimes.com,100.0,True
1,111678477449,2020-11-23 23:59:58,2020-11-25 18:17:53,Trump Administration To Begin Biden Transition Protocols,npr.org,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,https://www.npr.org/2020/11/23/937956178/trump-administration-to-begin-biden-transition-protocols?utm_source=facebook.com&utm_term=nprnews&utm_campaign=npr&utm_medium=social,https://www.facebook.com/OccupySpokane/posts/3477341109030911,21775,2.333333,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0,npr.org,100.0,True
2,111737873253,2020-11-24 23:59:23,2020-11-25 16:15:55,"Russia, eyeing foreign markets, says Sputnik COVID-19 vaccine to cost under $20",reuters.com,https://reut.rs/39bWCNG,https://reut.rs/39bWCNG,https://www.facebook.com/groups/335017933691230/permalink/962329240960093,18498,-21.000000,4619429,Council Member Ed Reece,edgarreece,2354,https://www.facebook.com/165175133572639,facebook_page,US,True,9,32,23,0,2,1,9,10,0,0,reut.rs,100.0,True
5,111622415061,2020-11-22 23:59:48,2020-11-25 18:11:01,This is a re-share of a post,NaN,https://www.businessinsider.com/china-emotional-surveillance-technology-2018-4,https://www.facebook.com/100576118560804/posts/100621611889588/,https://www.facebook.com/groups/378361242549568/permalink/1253075498411467,388,-0.750000,12571754,Australia & New Zealand Geoengineering Watch,NaN,388,https://www.facebook.com/groups/378361242549568,facebook_group,NaN,False,103,11,3,15,0,1,0,0,0,0,businessinsider.com,100.0,True
8,111622415061,2020-11-22 23:59:48,2020-11-25 18:11:01,This is a re-share of a post,NaN,https://www.wsj.com/articles/chinas-efforts-to-lead-the-way-in-ai-start-in-its-classrooms-11571958181,https://www.facebook.com/100576118560804/posts/100621611889588/,https://www.facebook.com/groups/378361242549568/permalink/1253075498411467,388,-0.750000,12571754,Australia & New Zealand Geoengineering Watch,NaN,388,https://www.facebook.com/groups/378361242549568,facebook_group,NaN,False,103,11,3,15,0,1,0,0,0,0,wsj.com,100.0,True


In [87]:
news_ratings_df_filtered.to_csv('news_ratings_df_filtered.csv', index=False)

In [92]:
domains_not_found = set(ratings_df.domain.unique()).difference(news_ratings_df_filtered.domain.unique())
print("#Domains not found in the dataset: ", len(domains_not_found))
print(domains_not_found)

40
{'stlpublicradio.org', 'rewire.news', 'tbo.com', 'tucsonnewsnow.com', 'policeone.com', 'citylab.com', 'whotv.com', 'dailytexanonline.com', 'streetsblog.org', 'mystatesman.com', 'citypages.com', 'foxnewsinsider.com', 'mydaytondailynews.com', 'onforb.es', 'myajc.com', 'msnewsnow.com', 'good4utah.com', 'splinternews.com', 'wday.com', 'watertowndailytimes.com', 'maps.yahoo.com', 'feeds.reuters.com', 'uk.businessinsider.com', 'drudgereport.com', 'siliconvalley.com', 'westernjournalism.com', 'ktuu.com', 'lasvegasnow.com', 'miami.com', 'ohio.com', 'news12.com', 'ljworld.com', 'rightwingnews.com', 'wttw.com', 'weeklystandard.com', 'add.my.yahoo.com', 'ktva.com', 'truth-out.org', 'thinkprogress.org', 'deseretnews.com'}
